###Cervical Cancer Risk Classification
This project does the resampling and validating of the model choosen with an imbalanced data provided.

In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, classification_report
from sklearn.model_selection import train_test_split

df = pd.read_csv ("/content/risk_factors_cervical_cancer.csv")

df['Number of sexual partners'].replace('?',df['Number of sexual partners'].mode()[0], inplace=True)
df['First sexual intercourse'].replace('?',df['First sexual intercourse'].mode()[0], inplace=True)
df['Num of pregnancies'].replace('?',df['Num of pregnancies'].mode()[0], inplace=True)
df['Smokes'].replace('?',df['Smokes'].mode()[0], inplace=True)
df['Smokes (years)'].replace('?',df['Smokes (years)'].mode()[0], inplace=True)
df['Smokes (packs/year)'].replace('?',df['Smokes (packs/year)'].mode()[0], inplace=True)
df['Hormonal Contraceptives'].replace('?',df['Hormonal Contraceptives'].mode()[0], inplace=True)
df['Hormonal Contraceptives (years)'].replace('?',df['Hormonal Contraceptives (years)'].mode()[0], inplace=True)
df['IUD'].replace('?',df['IUD'].mode()[0], inplace=True)
df['IUD (years)'].replace('?',df['IUD (years)'].mode()[0], inplace=True)
df['STDs'].replace('?',df['STDs'].mode()[0], inplace=True)
df['STDs (number)'].replace('?',df['STDs (number)'].mode()[0], inplace=True)
df['condylomatosis'].replace('?',df['condylomatosis'].mode()[0], inplace=True)
df['cervical condylomatosis'].replace('?',df['cervical condylomatosis'].mode()[0], inplace=True)
df['vaginal condylomatosis'].replace('?',df['vaginal condylomatosis'].mode()[0], inplace=True)
df['vulvo-perineal condylomatosis'].replace('?',df['vulvo-perineal condylomatosis'].mode()[0], inplace=True)
df['syphilis'].replace('?',df['syphilis'].mode()[0], inplace=True)
df['pelvic inflammatory disease'].replace('?',df['pelvic inflammatory disease'].mode()[0], inplace=True)
df['genital herpes'].replace('?',df['genital herpes'].mode()[0], inplace=True)
df['molluscum contagiosum'].replace('?',df['molluscum contagiosum'].mode()[0], inplace=True)
df['AIDS'].replace('?',df['AIDS'].mode()[0], inplace=True)
df['HIV'].replace('?',df['HIV'].mode()[0], inplace=True)
df['Hepatitis B'].replace('?',df['Hepatitis B'].mode()[0], inplace=True)
df['HPV'].replace('?',df['HPV'].mode()[0], inplace=True)
df[' Number of diagnosis'].replace('?',df[' Number of diagnosis'].mode()[0], inplace=True)

df = df.drop('Time since first diagnosis',axis=1).dropna()
df = df.drop('Time since last diagnosis',axis=1).dropna()
#print (df.columns)

print(df.columns[df.isin(['?']).any()])

X = df.drop('Cancer', axis=1).dropna()

y = df['Cancer'].dropna()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
random_forest = RandomForestClassifier(n_estimators=500).fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

# Accuracy vs Recall
print('Accuracy score: ' + str(accuracy_score(y_test, y_pred)))
print('Recall score: ' + str(recall_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))


Index([], dtype='object')
Accuracy score: 0.9813953488372092
Recall score: 0.5
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       209
           1       0.75      0.50      0.60         6

    accuracy                           0.98       215
   macro avg       0.87      0.75      0.80       215
weighted avg       0.98      0.98      0.98       215



# New Section

In [20]:
# Checking the Accuracy and Recall with only two columns
X = df.loc[:,['Smokes', 'Hormonal Contraceptives']]
y = df['Cancer'].dropna()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
random_forest = RandomForestClassifier(n_estimators=500).fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
print('Accuracy score: ' + str(accuracy_score(y_test, y_pred)))
print('Recall score: ' + str(recall_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

Accuracy score: 0.986046511627907
Recall score: 0.0
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       212
           1       0.00      0.00      0.00         3

    accuracy                           0.99       215
   macro avg       0.49      0.50      0.50       215
weighted avg       0.97      0.99      0.98       215



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Resampling

In [0]:
# Import the resampling package
from sklearn.utils import resample

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
# Returning to one dataframe
training_set = pd.concat([X_train, y_train], axis=1)
# Separating classes
cancer = training_set[training_set.Cancer == 1]
not_cancer = training_set[training_set.Cancer == 0]
#print('cancer:',cancer['Cancer'])
#print('not_cancer:',not_cancer['Cancer'])

Undersampling the Majority

In [22]:
# Undersampling the majority
undersample = resample(not_cancer, 
                       replace=True, 
                       n_samples=len(cancer), #set the number of samples to equal the number of the minority class
                       random_state=42)
# Returning to new training set
undersample_train = pd.concat([cancer, undersample])
undersample_train.Cancer.value_counts(normalize=True)

1    0.5
0    0.5
Name: Cancer, dtype: float64

In [23]:
# Testing on Random Forest classifier.

# Separate undersampled data into X and y sets
undersample_x_train = undersample_train.drop('Cancer', axis=1)
undersample_y_train = undersample_train.Cancer

# Fit model on undersampled data
undersample_rf = RandomForestClassifier(n_estimators=500).fit(undersample_x_train, undersample_y_train)

# Make predictions on test sets
y_pred = random_forest.predict(X_test)
print('Accuracy score: ' + str(accuracy_score(y_test, y_pred)))
print('Average Recall score: ' + str(recall_score(y_test, y_pred, average='macro')))
print(classification_report(y_test, y_pred))

Accuracy score: 0.986046511627907
Average Recall score: 0.5
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       212
           1       0.00      0.00      0.00         3

    accuracy                           0.99       215
   macro avg       0.49      0.50      0.50       215
weighted avg       0.97      0.99      0.98       215



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Oversampling the Minority

In [24]:
# Oversampling the minority
oversample = resample(cancer, 
                       replace=True, 
                       n_samples=len(not_cancer), #set the number of samples to equal the number of the majority class
                       random_state=42)

# Returning to new training set
oversample_train = pd.concat([not_cancer, oversample])
oversample_train.Cancer.value_counts(normalize=True)

1    0.5
0    0.5
Name: Cancer, dtype: float64

In [25]:
# Separate oversampled data into X and y sets
oversample_x_train = oversample_train.drop('Cancer', axis=1)
oversample_y_train = oversample_train.Cancer

# Fit model on oversampled data
oversample_rf = RandomForestClassifier(n_estimators=500).fit(oversample_x_train, oversample_y_train)

# Make predictions on test sets
y_pred = oversample_rf.predict(X_test)
print('Accuracy score: ' + str(accuracy_score(y_test, y_pred)))
print('Average Recall score: ' + str(recall_score(y_test, y_pred, average='macro')))
print(classification_report(y_test, y_pred))

Accuracy score: 0.6093023255813953
Average Recall score: 0.4732704402515723
              precision    recall  f1-score   support

           0       0.98      0.61      0.76       212
           1       0.01      0.33      0.02         3

    accuracy                           0.61       215
   macro avg       0.50      0.47      0.39       215
weighted avg       0.97      0.61      0.75       215



SMOTE (synthetic minority oversampling technique)

In [26]:
# Import the SMOTE package
from imblearn.over_sampling import SMOTE

# Synthesize minority class datapoints using SMOTE
sm = SMOTE(random_state=42, sampling_strategy='minority')
smote_x_train, smote_y_train = sm.fit_resample(X_train, y_train)

# Separate into training and test sets
smote_x_train = pd.DataFrame(smote_x_train, columns = X_train.columns)
smote_y_train = pd.DataFrame(smote_y_train, columns = ['Cancer'])
smote = RandomForestClassifier(n_estimators=1000).fit(smote_x_train, smote_y_train)
 
# Predict on training set
smote_preds = smote.predict(X_test)

# Checking accuracy and recall
print('Accuracy Score: ', accuracy_score(y_test, smote_preds),'\n\n')
print('Averaged Recall Score: ', recall_score(y_test, smote_preds, average='macro'), '\n\n')
    
print(classification_report(y_test, smote_preds))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


Accuracy Score:  0.4232558139534884 


Averaged Recall Score:  0.5432389937106918 


              precision    recall  f1-score   support

           0       0.99      0.42      0.59       212
           1       0.02      0.67      0.03         3

    accuracy                           0.42       215
   macro avg       0.50      0.54      0.31       215
weighted avg       0.98      0.42      0.58       215

